Integrante:

- Morales López Erik Brandon (20163041)

In [1]:
# importing the libraries
import pandas as pd
import numpy as np
import pyreadr

# Loading Data

In [2]:
ruta="https://raw.githubusercontent.com/alexanderquispe/ECO224/main/Labs/data/penn_jae.dat"

## loading the data
Penn = pd.read_csv(ruta , sep='\s', engine='python')
#dimension 
n = Penn.shape[0]
#number of covariables 
p_1 = Penn.shape[1]
Penn = Penn[ (Penn['tg'] == 4) | (Penn['tg'] == 0) ]

In [3]:
Penn.columns
Penn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5099 entries, 0 to 13911
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   abdt         5099 non-null   int64  
 1   tg           5099 non-null   int64  
 2   inuidur1     5099 non-null   int64  
 3   inuidur2     5099 non-null   int64  
 4   female       5099 non-null   int64  
 5   black        5099 non-null   int64  
 6   hispanic     5099 non-null   int64  
 7   othrace      5099 non-null   int64  
 8   dep          5099 non-null   int64  
 9   q1           5099 non-null   int64  
 10  q2           5099 non-null   int64  
 11  q3           5099 non-null   int64  
 12  q4           5099 non-null   int64  
 13  Unnamed: 13  5099 non-null   int64  
 14  q5           5099 non-null   int64  
 15  q6           5099 non-null   int64  
 16  recall       5099 non-null   int64  
 17  agelt35      5099 non-null   int64  
 18  agegt54      5099 non-null   int64  
 19  durab

# Some changes to the data

In [4]:

Penn['T4'] = (Penn[['tg']]==4).astype(int)

# Create category variable
Penn['dep'] = Penn['dep'].astype( 'category' )
Penn.head()

,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1,...,q6,recall,agelt35,agegt54,durable,nondurable,lusd,husd,muld,T4
0,10824,0,18,18,0,0,0,0,2,0,...,0,0,0,0,0,0,1,0,NaN,0
3,10824,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,NaN,0
4,10747,0,27,27,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,NaN,0
11,10607,4,9,9,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,NaN,1
12,10831,0,27,27,0,0,0,0,1,0,...,0,0,1,1,0,1,0,0,NaN,0


In [5]:
Penn['dep'].unique()

[2, 0, 1]
Categories (3, int64): [0, 1, 2]

In [6]:
# Im creating a new column called log_inuidor1 which is the log from inuidur1
Penn["log_inuidor1"]=np.log(Penn["inuidur1"])
Penn.head()

,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1,...,recall,agelt35,agegt54,durable,nondurable,lusd,husd,muld,T4,log_inuidor1
0,10824,0,18,18,0,0,0,0,2,0,...,0,0,0,0,0,1,0,NaN,0,2.890372
3,10824,0,1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,NaN,0,0.000000
4,10747,0,27,27,0,0,0,0,0,0,...,0,0,0,0,1,0,0,NaN,0,3.295837
11,10607,4,9,9,0,0,0,0,0,0,...,1,0,0,0,0,0,1,NaN,1,2.197225
12,10831,0,27,27,0,0,0,0,1,0,...,0,1,1,0,1,0,0,NaN,0,3.295837


# Bootstrapping

In [7]:

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error
import statsmodels.formula.api as smf
import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
Penn_1=Penn.copy()
Penn_1.reset_index(inplace=True)
Penn_1.head()

,index,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,...,recall,agelt35,agegt54,durable,nondurable,lusd,husd,muld,T4,log_inuidor1
0,0,10824,0,18,18,0,0,0,0,2,...,0,0,0,0,0,1,0,NaN,0,2.890372
1,3,10824,0,1,1,0,0,0,0,0,...,0,0,0,0,1,0,0,NaN,0,0.000000
2,4,10747,0,27,27,0,0,0,0,0,...,0,0,0,0,1,0,0,NaN,0,3.295837
3,11,10607,4,9,9,0,0,0,0,0,...,1,0,0,0,0,0,1,NaN,1,2.197225
4,12,10831,0,27,27,0,0,0,0,1,...,0,1,1,0,1,0,0,NaN,0,3.295837


In [9]:
def get_indices(data,num_samples):
    return  np.random.choice(data.index, num_samples, replace=True)

In [10]:
index=get_indices(Penn_1,1000)
index

array([ 148, 3856, 3739,  682, 2934, 1213, 1562, 1218, 1080, 3370, 1503,
       1949, 3636, 2569,  820, 1944,  644, 2127, 4193, 3409,  865, 1278,
       1626, 2845, 1083, 4826, 3865, 2827,  371,  477, 3312, 2156, 4673,
        901, 2175, 4862, 3712, 1538, 2286,  290, 1501, 2236, 4715, 4111,
        836, 4985, 4674, 3727, 3491,  609, 2703, 1942, 2400, 2229, 4637,
       1547,  534, 4166, 1374,  440, 2622, 2584,  268, 1687, 3486, 1268,
       1985, 4643, 1946, 3163, 4649, 3417, 4367, 4501, 1208, 2147, 4573,
       2521, 2493, 3112, 2653, 2748,  556, 1111, 3622, 3064, 4780, 4669,
       1886, 3048, 3365, 1269, 3204, 3916,  102,  346, 4616,  266, 3211,
       3965, 1802, 4630, 2029, 4836,  349, 1602, 4171,  231, 2173, 1701,
       2181, 3059, 4609, 3657,  669, 3070, 2192, 1379, 3407, 3467, 2574,
       2425, 4157, 1634,  945, 2042, 4521, 1874, 1884, 5044,  239,  238,
       1705,  331, 2394, 5030, 3705, 4942, 4326, 3951, 1017, 2352, 1549,
       4350, 3378, 3696, 3614, 4543, 1376, 4205, 20

In [11]:
# similar to boot.fn in lab

def get_estimates(Penn_1,index):
    X = Penn_1[["T4","female","black","othrace","dep","q2","q3","q4","q5","q6","agelt35","agegt54","durable","lusd",
               "husd"]].loc[index]
    y = Penn_1["log_inuidor1"].loc[index]
    
    lr = LinearRegression()
    lr.fit(X,y)
    intercept = lr.intercept_
    coef = lr.coef_
    return [intercept,coef]

* at this point, I divide the data according the lab. on X variable I am using the *T4*, *female* and *Black* and more, but on Y variable Im using only the *log_inuidor1*

In [12]:
#time to build the bootstrap technique, also Im adding a new term which is coeff_3, because I want to extract the first variables with its coeff and standar erros.
def boot(Penn_1,func,R):
    intercept = []
    coeff_1 = []
    coeff_2 = []
    coeff_3 = []
    for i in range(R):
        intercept.append(func(Penn_1, get_indices(Penn_1,1000))[0])
        coeff_1.append(func(Penn_1, get_indices(Penn_1,1000))[1][0])
        coeff_2.append(func(Penn_1, get_indices(Penn_1,1000))[1][1])
        coeff_3.append(func(Penn_1, get_indices(Penn_1,1000))[1][2])
    intercept_statistics = {'estimated_value':np.mean(intercept),'std_error':np.std(intercept)}   
    coeff_1_statistices = {'estimated_value':np.mean(coeff_1),'std_error':np.std(coeff_1)}
    coeff_2_statistices = {'estimated_value':np.mean(coeff_2),'std_error':np.std(coeff_2)}
    coeff_3_statistices = {'estimated_value':np.mean(coeff_3),'std_error':np.std(coeff_3)}
    return {'intercept':intercept_statistics,'coeff_1_statistices':coeff_1_statistices,
            'coeff_2_statistices':coeff_2_statistices,'coeff_3_statistices':coeff_3_statistices}

In [13]:
results = boot(Penn_1,get_estimates,1000)

In [14]:
print('Result for intercept ',results['intercept'])
print('Result for coefficient term 1',results['coeff_1_statistices'])
print('Result for coefficient term 2',results['coeff_2_statistices'])
print('Result for coefficient term 3',results['coeff_3_statistices'])

Result for intercept  {'estimated_value': 1.7711316030665167, 'std_error': 0.11424702463769908}
Result for coefficient term 1 {'estimated_value': -0.07459417487813234, 'std_error': 0.07923687354483962}
Result for coefficient term 2 {'estimated_value': 0.13677199098207513, 'std_error': 0.0800542564278317}
Result for coefficient term 3 {'estimated_value': -0.31476535119665416, 'std_error': 0.13571899777754406}


In [17]:
new_data=pd.DataFrame({
    "T4":[-0.07,0.08],
    "female":[0.13,0.07],
    "black":[-0.30,0.13], 
    "index":["coef","std_error"]
})

# Here is the result from the Bootstrap

In [18]:
# I created a new data.frame to show the results
new_data.set_index("index")

,T4,female,black
index,,,
coef,-0.07,0.13,-0.30
std_error,0.08,0.07,0.13


* As can be seen in the table I have created, it is understood that I have made 1000 bootstrap estimates for the variables indicated in the exercise such a **T4** = -0.07, **female** = 0.13 and **black** = -0.30. Making a comparison with the OLS model and the bootstrap we come to the conclusion that the coefficients remain the same or rather there is not much variability, however, what if there is variability is in the standard errors of the bootsrap and the OLS.

In [35]:
#using statmodels

formula="log_inuidor1 ~ T4 + (female+black+othrace+dep+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)"

reg1=smf.ols(formula=formula, data=Penn_1).fit()

#results
print(reg1.summary())

                            OLS Regression Results                            
Dep. Variable:           log_inuidor1   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     12.51
Date:                Thu, 11 Nov 2021   Prob (F-statistic):           3.92e-33
Time:                        15:20:55   Log-Likelihood:                -8128.2
No. Observations:                5099   AIC:                         1.629e+04
Df Residuals:                    5082   BIC:                         1.640e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.7723      0.049     36.011      0.0